In [15]:
import os
import numpy as np

import tables

from tables import *

# Declaring a Column Descriptor
Now, imagine that we have a particle detector and we want to create a table object in order to save data retrieved from it. You need first to define the table, the number of columns it has, what kind of object is contained in each column, and so on.

Our particle detector has a TDC (Time to Digital Converter) counter with a dynamic range of 8 bits and an ADC (Analogical to Digital Converter) with a range of 16 bits. For these values, we will define 2 fields in our record object called TDCcount and ADCcount. We also want to save the grid position in which the particle has been detected, so we will add two new fields called grid_i and grid_j. Our instrumentation also can obtain the pressure and energy of the particle. The resolution of the pressure-gauge allows us to use a single-precision float to store pressure readings, while the energy value will need a double-precision float. Finally, to track the particle we want to assign it a name to identify the kind of the particle it is and a unique numeric identifier. So we will add two more fields: name will be a string of up to 16 characters, and idnumber will be an integer of 64 bits (to allow us to store records for extremely large numbers of particles).

Having determined our columns and their types, we can now declare a new Particle class that will contain all this information:

In [16]:
class Particle(IsDescription):
    name      = StringCol(16)   # 16-character String
    idnumber  = Int64Col()      # Signed 64-bit integer
    ADCcount  = UInt16Col()     # Unsigned short integer
    TDCcount  = UInt8Col()      # unsigned byte
    grid_i    = Int32Col()      # 32-bit integer
    grid_j    = Int32Col()      # 32-bit integer
    pressure  = Float32Col()    # float  (single-precision)
    energy    = Float64Col()    # double (double-precision)

This definition class is self-explanatory. Basically, you declare a class variable for each field you need. As its value you assign an instance of the appropriate Col subclass, according to the kind of column defined (the data type, the length, the shape, etc). See the The Col class and its descendants for a complete description of these subclasses. See also Supported data types in PyTables for a list of data types supported by the Col constructor.

From now on, we can use Particle instances as a descriptor for our detector data table. We will see later on how to pass this object to construct the table. But first, we must create a file where all the actual data pushed into our table will be saved.

# Creating a PyTables file from scratch
Use the top-level **`open_file()`** function to create a PyTables file:

In [22]:
h5file = open_file(
    os.path.join('tmp', 'tutorial1.h5'), 
    mode="w", title="Test file")
h5file.close()

**`open_file()`** is one of the objects imported by the **`from tables import *`** statement. Here, we are saying that we want to create a new file in the current working directory called “tutorial1.h5” in “w”rite mode and with an descriptive title string (“Test file”). This function attempts to open the file, and if successful, returns the File (see The File Class) object instance h5file. The root of the object tree is specified in the instance’s root attribute.

# Creating a new group
Now, to better organize our data, we will create a group called detector that branches from the root node. We will save our particle data table in this group:

In [24]:
group = h5file.create_group("/", 'detector', 'Detector information')

ClosedFileError: the file object is closed

Here, we have taken the File instance h5file and invoked its File.create_group() method to create a new group called detector branching from “/” (another way to refer to the h5file.root object we mentioned above). This will create a new Group (see The Group class) object instance that will be assigned to the variable group.

# Creating a new table
Let’s now create a Table (see The **[Table class](https://www.pytables.org/usersguide/libref/structured_storage.html#tableclassdescr)**) object as a branch off the newly-created group. We do that by calling the **`File.create_table()`** method of the h5file object:

In [25]:
table = h5file.create_table(group, 'readout', Particle, "Readout example")

NameError: name 'group' is not defined

We create the Table instance under group. We assign this table the node name “readout”. The Particle class declared before is the description parameter (to define the columns of the table) and finally we set “Readout example” as the Table title. With all this information, a new Table instance is created and assigned to the variable table.

If you are curious about how the object tree looks right now, simply print the File instance variable h5file, and examine the output:

In [26]:
print(h5file)

<closed File>


As you can see, a dump of the object tree is displayed. It’s easy to see the Group and Table objects we have just created. If you want more information, just type the variable containing the File instance:

In [27]:
h5file

<closed File>